https://sorabatake.jp/11731/

In [3]:
import numpy as np 
import os
import random 
#擬似乱数を生成するライブラリー 
from PIL import Image, ImageOps 
#画像処理ライブラリー
from sklearn.metrics import accuracy_score 
#モデルの正解率を計算するライブラリー
import glob 
#ファイルパス名を取得するライブラリ
import matplotlib.pyplot as plt 
#図をプロットするためのライブラリー
%matplotlib inline

DATA0 = "../snap/density/5/*"#コネクションなし
DATA1 = "../snap/density/7/*"#コネクションありに変える
DATA1TEST ="..snap/density/3/*"
PATH1TEST ="..snap/density/8/*"
ALTIMAGES0 = "../imgout/MLtest/0/"
ALTIMAGES1 = "../imgout/MLtest/1/"
IMGSHAPE = (128,128)

np.random.seed(0) 
 #乱数のシードの設定

# 水増し

In [ ]:
#雲がない画像ファイルのパスのリストを取得
files = glob.glob(DATA0) 

#出力ディレクトリのパス
temp_output_dir = ALTIMAGES0

#ランダムな領域の切り取りを行う関数
def random_crop(image, size=0.8):
    image = np.array(image, dtype=np.float64)

    height, width, _ = image.shape
    crop_size = int(min(height, width) * size)

    top = np.random.randint(0, height - crop_size)
    left = np.random.randint(0, width - crop_size)
    bottom = top + crop_size
    right = left + crop_size
    image = image[top:bottom, left:right,:]

    return Image.fromarray(np.uint8(image))


def altimg_save(item):    
    img = Image.open(item)
    file_name = os.path.basename(item)[:-4] 
    
    img_flip = ImageOps.flip(img) # 画像の上下反転
    img_flip.save(temp_output_dir + file_name + "_flip.png") # 画像保存
        
    img_mirror = ImageOps.mirror(img) # 画像の左右反転
    img_mirror.save(temp_output_dir + file_name + "_mirror.png") # 画像保存
            
    img_rotate = img.rotate(90) # 画像の90度回転
    img_rotate.save(temp_output_dir + file_name + "_rot90.png") # 画像保存

    img_crop = random_crop(img) # 画像の切り取り
    img_crop = img_crop.resize((256, 256)) # 元のサイズに戻す
    img_crop.save(temp_output_dir + file_name + "_crop.png") # 画像保存
for item in files:
    
    altimg_save(item)

In [ ]:
#雲がある画像ファイルのパスのリストを取得
files = glob.glob(DATA1) 

#出力ディレクトリのパス
temp_output_dir = ALTIMAGES1

for item in files:
    altimg_save(item)


# 読み込み

In [ ]:
# 訓練データ
num_of_data_clear = len(glob.glob(DATA0)) # 雲がない画像の枚数
num_of_data_cloudy = len(glob.glob(DATA1)) # 雲がある画像の枚数
num_of_data_total = num_of_data_clear + num_of_data_cloudy # 学習データの全枚数

N_col = IMGSHAPE[0]*IMGSHAPE[1]*3 # 行列の列数
X_train = np.zeros((num_of_data_total, N_col)) # 学習データ格納のためゼロ行列生成
y_train = np.zeros((num_of_data_total)) # 学習データに対するラベルを格納するためのゼロ行列生成

# 雲がない画像を行列に読み込む
path_list = glob.glob(ALTIMAGES0) 
i_count = 0

for item in path_list:
    im = Image.open(item).convert('RGB')
    img_resize =im.resize(IMGSHAPE)  # 画像のサイズ変更
    im_array = np.ravel(np.asarray(img_resize)) # 画像を配列に変換
    im_regularized = im_array/255. # 正規化
    X_train[i_count,:] =  im_regularized 
    y_train[i_count] = 0 # 雲がないことを表すラベル
    i_count += 1
  
# 雲がある画像を行列に読み込む
path_list = glob.glob(ALTIMAGES1) 

for item in path_list:
    im = Image.open(item).convert('RGB')   
    img_resize = im.resize(IMGSHAPE) # 画像のサイズ変更
    im_array = np.ravel(np.asarray(img_resize)) # 画像を配列に変換
    im_regularized = im_array/255. # 正規化
    X_train[i_count,:] = im_regularized
    y_train[i_count] = 1 # 雲があることを表すラベル
    i_count += 1

In [ ]:
# テストデータ
num_of_data_clear = len(glob.glob(DATA1TEST)) # 雲がない画像の枚数
num_of_data_cloudy = len(glob.glob(PATH1TEST)) # 雲がある画像の枚数
num_of_data_total = num_of_data_clear + num_of_data_cloudy # テストデータの全枚数

N_col = IMGSHAPE[0]*IMGSHAPE[1]*3 # 行列の列数(RGBなら*3)
X_test = np.zeros((num_of_data_total, N_col)) # テストデータ格納のためゼロ行列生成
y_test = np.zeros(num_of_data_total) # テストデータに対するラベルを格納するためのゼロ行列生成

# 雲がない画像を行列に読み込む
path_list = glob.glob("./Dataset_AVNIR2/test/clear/*") 
i_count = 0
for item in path_list:
    im = Image.open(item).convert('RGB')
    img_resize =im.resize(IMGSHAPE) # 画像のサイズ変更
    im_array = np.ravel(np.array(img_resize)) # 画像を配列に変換
    im_mean_subtract = im_array/255. # 正規化 
    X_test[i_count,:] =  im_mean_subtract
    y_test[i_count] = 0 # 雲がないことを表すラベル
    i_count += 1

# 雲がある画像を行列に読み込む
path_list = glob.glob(PATH1TEST) 

for item in path_list:
    im = Image.open(item).convert('RGB')   
    img_resize = im.resize(IMGSHAPE) # 画像のサイズ変更
    im_array = np.ravel(np.array(img_resize)) # 画像を配列に変換
    im_regularized = im_array/255. # 正規化
    X_test[i_count,:] = im_regularized  
    y_test[i_count] = 1 # 雲があることを表すラベル
    i_count += 1 

# 学習

# PCA

In [ ]:
from sklearn.decomposition import PCA
N_dim =  100 # 49152(=128×128×3)の列を100列に落とし込む

pca = PCA(n_components=N_dim, random_state=0)

X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

print('累積寄与率: {0}'.format(sum(pca.explained_variance_ratio_)))

# Linear SVC

In [ ]:
from sklearn.svm import LinearSVC # ライブラリのインポート

model = LinearSVC(C=0.3, random_state=0) # インスタンスを生成

model.fit(X_train_pca, y_train) # モデルの学習

# 学習データに対する精度
print("Train :", model.score(X_train_pca,  y_train)) 

# テストデータに対する精度
print("Test :", model.score(X_test_pca, y_test)) 

# KNeighbors Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier　# ライブラリのインポート

n_neighbors = int(np.sqrt(6000))  # kの設定

model = KNeighborsClassifier(n_neighbors = n_neighbors)  
model.fit(X_train_pca, y_train)　# モデルの学習
    
# 精度
print("Train :", model.score(X_train_pca, y_train))
print("Test :", model.score(X_test_pca, y_test))

# 非線形SVC RBFカーネル

In [ ]:

from sklearn.svm import SVC　# ライブラリのインポート

model = SVC(C=0.3, kernel='rbf', random_state=0)　# インスタンスを生成 

model.fit(X_train_pca, y_train)　# モデルの学習

# 精度
print("Train :", model.score(X_train_pca,  y_train))
print("Test :", model.score(X_test_pca, y_test))

# XGBoost

In [ ]:
import xgboost as xgb # ライブラリのインポート

model = xgb.XGBClassifier(n_estimators=80, max_depth=4, gamma=3) # インスタンスの生成
           
model.fit(X_train_pca, y_train) # モデルの学習
   
# 精度
print("Train :", model.score(X_train_pca,  y_train))
print("Test :", model.score(X_test_pca, y_test))

# まとめ

In [ ]:
train_sizes = np.array([0.1, 0.33, 0.55, 0.78, 1. ]) # 用いる学習データの割合を全体の0.1〜1まで変化させる

train_sizes, train_scores_linearSVC, test_scores_linearSVC = learning_curve(
    model_linearSVC, X_train_pca, y_train, cv=5, train_sizes=train_sizes, random_state=0, shuffle=True
    ) # 様々学習データ数に対して、モデルを学習・評価する。

# 各学習データ数の場合について学習用データと検証用データに対する精度を求める。
train_scores_mean_linearSVC = np.mean(train_scores_linearSVC, axis=1)
train_scores_std_linearSVC = np.std(train_scores_linearSVC, axis=1)
validation_scores_mean_linearSVC = np.mean(validation_scores_linearSVC, axis=1)
validation_scores_std_linearSVC = np.std(validation_scores_linearSVC, axis=1)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 8))
plt.subplots_adjust(wspace=0.4, hspace=0.6)

# Linear SVCの学習曲線
ax[0,0].plot(train_sizes, train_scores_mean_linearSVC, 'o-', color="r", label="Training score")
ax[0,0].plot(train_sizes, validation_scores_mean_linearSVC, 'o-', color="g", label="Test score")

ax[0, 0].set_title('Linear SVC')
ax[0, 0].set_xlabel('Number of Taining data')
ax[0, 0].set_ylabel('Accyracy')
ax[0,0].set_ylim(0.4, 0.8)
ax[0,0].legend(loc="best")

# kNNの学習曲線
ax[1,0].plot(train_sizes, train_scores_mean_kNN, 'o-', color="r", label="Training score")
ax[1,0].plot(train_sizes, validation_scores_mean_kNN, 'o-', color="g", label="Test score")

ax[1, 0].set_title('k-NN')
ax[1, 0].set_xlabel('Number of Taining data')
ax[1, 0].set_ylabel('Accuracy')
ax[1,0].set_ylim(0.5, 0.7)
ax[1,0].legend(loc="best")

# 非線形SVCの学習曲線
ax[0,1].plot(train_sizes, train_scores_mean_nonlinearSVC, 'o-', color="r", label="Training score")
ax[0,1].plot(train_sizes, validation_scores_mean_nonlinearSVC, 'o-', color="g", label="Test score")
ax[0, 1].set_title('non-linear SVC')
ax[0, 1].set_xlabel('Number of Taining data')
ax[0, 1].set_ylabel('Accuracy')
ax[0,1].set_ylim(0.7, 0.9)
ax[0,1].legend(loc="best")

# XGBoostの学習曲線
ax[1,1].plot(train_sizes, train_scores_mean_xgb, 'o-', color="r", label="Training score")
ax[1,1].plot(train_sizes, validation_scores_mean_xgb, 'o-', color="g", label="Test score")

ax[1, 1].set_title('XGBoost')
ax[1, 1].set_xlabel('Number of Taining data')
ax[1, 1].set_ylabel('Accuracy')
ax[1,1].set_ylim(0.7, 1.0)
ax[1,1].legend(loc="best")

In [7]:
import matplotlib.pyplot as plt
from mymodule import myfunc as mf
vX = mf.load(mf.gen_snap_path("velocityX",1,1),z=3)

C:/Users/lkxl1/OneDrive/デスクトップ/research/bigendian_data/


In [15]:
img = Image.open("../snap/velocityX/01/velocityX.01.01")
img_rotate = img.rotate(90)

UnidentifiedImageError: cannot identify image file '../snap/velocityX/01/velocityX.01.01'

In [13]:
mf.gen_snap_path("velocityX",1,1)

'C:/Users/lkxl1/OneDrive/デスクトップ/research/snap/velocityX/01/velocityX.01.01'

In [17]:
img = Image.open("../imgout/1005/png/density.1.png")

In [19]:
img.type

AttributeError: 'PngImageFile' object has no attribute 'type'